# Trying out different BERT models

Through this notebook, my goal was to try out different BERT based models to see which one performs the best. I ended up choosing the BERT-BASE model due to its compact nature and comparable performace to BERT-LEARGE and ROBERTA.

## BERT BASE

In [5]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
import pickle

# Load the dataset
df = pd.read_csv('/kaggle/input/no-negative-transactions/amex_all_categorized_raw.csv')
df = df[df['Description'] != 'Description'] 

# Encode categories
label_encoder = LabelEncoder()
df['Category_encoded'] = label_encoder.fit_transform(df['Category'])

# Save the encoder
with open('/kaggle/working/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
input_ids = []
attention_masks = []

for desc in df['Description']:
    encoded_dict = tokenizer.encode_plus(
        desc,                              # Sentence to encode
        add_special_tokens = True,         # Add '[CLS]' and '[SEP]'
        max_length = 64,                   # Pad & truncate all sentences
        pad_to_max_length = True,
        return_attention_mask = True,      # Construct attention masks
        return_tensors = 'pt',             # Return pytorch tensors
    )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['Category_encoded'].values)

# Splitting the dataset
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

# Convert to DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, batch_size=32)


# Define the custom model
class BertForSequenceClassificationCustom(nn.Module):
    def __init__(self, num_labels):
        super(BertForSequenceClassificationCustom, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the model
num_labels = len(df['Category'].unique())
model = BertForSequenceClassificationCustom(num_labels)

# Define the evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
        
        logits = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

        total_eval_accuracy += np.sum(predictions == labels_flat) / len(labels_flat)
    
    return total_eval_accuracy / len(dataloader)

# Before fine-tuning, evaluate the model to get the initial accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

initial_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Initial Validation Accuracy: {initial_accuracy:.4f}')

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 25  # Adjust the number of epochs here
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
for epoch_i in range(0, epochs):  # Loop through 15 epochs
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        logits = model(b_input_ids, attention_mask=b_input_mask)
        
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, b_labels)
        
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f" Average training loss: {avg_train_loss:.2f}")

# After fine-tuning, evaluate the model again to get the new accuracy
final_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Final Validation Accuracy: {final_accuracy:.4f}')

model_save_path = "/kaggle/working/BERT_BASE_ft_model.pth"
torch.save(model.state_dict(), model_save_path)

print("Training complete")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Initial Validation Accuracy: 0.0078
======== Epoch 1 / 25 ========


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 Average training loss: 2.49
======== Epoch 2 / 25 ========
 Average training loss: 2.01
======== Epoch 3 / 25 ========
 Average training loss: 1.70
======== Epoch 4 / 25 ========
 Average training loss: 1.45
======== Epoch 5 / 25 ========
 Average training loss: 1.22
======== Epoch 6 / 25 ========
 Average training loss: 1.00
======== Epoch 7 / 25 ========
 Average training loss: 0.81
======== Epoch 8 / 25 ========
 Average training loss: 0.65
======== Epoch 9 / 25 ========
 Average training loss: 0.55
======== Epoch 10 / 25 ========
 Average training loss: 0.46
======== Epoch 11 / 25 ========
 Average training loss: 0.39
======== Epoch 12 / 25 ========
 Average training loss: 0.35
======== Epoch 13 / 25 ========
 Average training loss: 0.30
======== Epoch 14 / 25 ========
 Average training loss: 0.28
======== Epoch 15 / 25 ========
 Average training loss: 0.26
======== Epoch 16 / 25 ========
 Average training loss: 0.22
======== Epoch 17 / 25 ========
 Average training loss: 0.22
===

## BERT LARGE

In [3]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn

# Load the dataset
df = pd.read_csv('/kaggle/input/no-negative-transactions/amex_all_categorized_raw.csv')
df = df[df['Description'] != 'Description']

# Encode categories
label_encoder = LabelEncoder()
df['Category_encoded'] = label_encoder.fit_transform(df['Category'])

# Initialize the tokenizer with bert-large-uncased
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Tokenize the dataset
input_ids = []
attention_masks = []

for desc in df['Description']:
    encoded_dict = tokenizer.encode_plus(
        desc,                              # Sentence to encode
        add_special_tokens = True,         # Add '[CLS]' and '[SEP]'
        max_length = 64,                   # Pad & truncate all sentences
        pad_to_max_length = True,
        return_attention_mask = True,      # Construct attention masks
        return_tensors = 'pt',             # Return pytorch tensors
    )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['Category_encoded'].values)

# Splitting the dataset
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

# Convert to DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, batch_size=32)

# Define the custom model with BertModel using bert-large-uncased
class BertForSequenceClassificationCustom(nn.Module):
    def __init__(self, num_labels):
        super(BertForSequenceClassificationCustom, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-large-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1024, num_labels) # Update the size for bert-large-uncased

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the model
num_labels = len(df['Category'].unique())
model = BertForSequenceClassificationCustom(num_labels)

# (The rest of the code remains unchanged)

# Define the evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
        
        logits = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

        total_eval_accuracy += np.sum(predictions == labels_flat) / len(labels_flat)
    
    return total_eval_accuracy / len(dataloader)

# Before fine-tuning, evaluate the model to get the initial accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

initial_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Initial Validation Accuracy: {initial_accuracy:.4f}')

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 25  # Adjust the number of epochs here
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
for epoch_i in range(0, epochs):  # Loop through 15 epochs
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        logits = model(b_input_ids, attention_mask=b_input_mask)
        
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, b_labels)
        
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f" Average training loss: {avg_train_loss:.2f}")

# After fine-tuning, evaluate the model again to get the new accuracy
final_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Final Validation Accuracy: {final_accuracy:.4f}')

model_save_path = "/kaggle/working/BERT_LARGE_ft_model.pth"
torch.save(model.state_dict(), model_save_path)

print("Training complete")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Initial Validation Accuracy: 0.0625
======== Epoch 1 / 25 ========


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 Average training loss: 2.42
======== Epoch 2 / 25 ========
 Average training loss: 2.05
======== Epoch 3 / 25 ========
 Average training loss: 1.63
======== Epoch 4 / 25 ========
 Average training loss: 1.15
======== Epoch 5 / 25 ========
 Average training loss: 0.86
======== Epoch 6 / 25 ========
 Average training loss: 0.66
======== Epoch 7 / 25 ========
 Average training loss: 0.49
======== Epoch 8 / 25 ========
 Average training loss: 0.41
======== Epoch 9 / 25 ========
 Average training loss: 0.34
======== Epoch 10 / 25 ========
 Average training loss: 0.27
======== Epoch 11 / 25 ========
 Average training loss: 0.25
======== Epoch 12 / 25 ========
 Average training loss: 0.21
======== Epoch 13 / 25 ========
 Average training loss: 0.19
======== Epoch 14 / 25 ========
 Average training loss: 0.19
======== Epoch 15 / 25 ========
 Average training loss: 0.18
======== Epoch 16 / 25 ========
 Average training loss: 0.16
======== Epoch 17 / 25 ========
 Average training loss: 0.16
===

## Roberta

In [4]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
import torch.nn as nn

# Load the dataset
df = pd.read_csv('/kaggle/input/no-negative-transactions/amex_all_categorized_raw.csv')
df = df[df['Description'] != 'Description']  # Clean any repeated headers

# Encode categories
label_encoder = LabelEncoder()
df['Category_encoded'] = label_encoder.fit_transform(df['Category'])

# Initialize the tokenizer with roberta-large
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

# Tokenize the dataset
input_ids = []
attention_masks = []

for desc in df['Description']:
    encoded_dict = tokenizer.encode_plus(
        desc,                              # Sentence to encode
        add_special_tokens = True,         # Add '[CLS]' and '[SEP]'
        max_length = 64,                   # Pad & truncate all sentences
        pad_to_max_length = True,
        return_attention_mask = True,      # Construct attention masks
        return_tensors = 'pt',             # Return pytorch tensors
    )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['Category_encoded'].values)

# Splitting the dataset
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

# Convert to DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, batch_size=32)

# Define the custom model with RobertaModel
class RobertaForSequenceClassificationCustom(nn.Module):
    def __init__(self, num_labels):
        super(RobertaForSequenceClassificationCustom, self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained('roberta-large')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1024, num_labels) # Ensure the size matches roberta-large's output features

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the model
num_labels = len(df['Category'].unique())
model = RobertaForSequenceClassificationCustom(num_labels)

# Define the evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
        
        logits = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

        total_eval_accuracy += np.sum(predictions == labels_flat) / len(labels_flat)
    
    return total_eval_accuracy / len(dataloader)

# Before fine-tuning, evaluate the model to get the initial accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

initial_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Initial Validation Accuracy: {initial_accuracy:.4f}')

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 25  # Adjust the number of epochs here
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
for epoch_i in range(0, epochs):  # Loop through 15 epochs
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        logits = model(b_input_ids, attention_mask=b_input_mask)
        
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, b_labels)
        
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f" Average training loss: {avg_train_loss:.2f}")

# After fine-tuning, evaluate the model again to get the new accuracy
final_accuracy = evaluate_model(model, validation_dataloader, device)
print(f'Final Validation Accuracy: {final_accuracy:.4f}')

model_save_path = "/kaggle/working/RoBERTa_LARGE_ft_model.pth"
torch.save(model.state_dict(), model_save_path)

print("Training complete")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initial Validation Accuracy: 0.0703
======== Epoch 1 / 25 ========


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 Average training loss: 2.28
======== Epoch 2 / 25 ========
 Average training loss: 1.57
======== Epoch 3 / 25 ========
 Average training loss: 0.96
======== Epoch 4 / 25 ========
 Average training loss: 0.67
======== Epoch 5 / 25 ========
 Average training loss: 0.51
======== Epoch 6 / 25 ========
 Average training loss: 0.38
======== Epoch 7 / 25 ========
 Average training loss: 0.32
======== Epoch 8 / 25 ========
 Average training loss: 0.28
======== Epoch 9 / 25 ========
 Average training loss: 0.24
======== Epoch 10 / 25 ========
 Average training loss: 0.22
======== Epoch 11 / 25 ========
 Average training loss: 0.18
======== Epoch 12 / 25 ========
 Average training loss: 0.18
======== Epoch 13 / 25 ========
 Average training loss: 0.17
======== Epoch 14 / 25 ========
 Average training loss: 0.17
======== Epoch 15 / 25 ========
 Average training loss: 0.16
======== Epoch 16 / 25 ========
 Average training loss: 0.14
======== Epoch 17 / 25 ========
 Average training loss: 0.13
===